In [9]:
import os
import torch
import numpy as np
import pandas as pd

from tqdm import tqdm

# Configuration

In [10]:
config = {
    'data_root': './data',
    'device': 'mps'
}

# Utils

In [19]:
from torch.utils.data import random_split

In [24]:
def train_valid_split(full_dataset, valid_ratio):
    train_set, valid_set = random_split(full_dataset, [1-valid_ratio, valid_ratio])
    return train_set, valid_set

# Dataset

In [25]:
from torch.utils.data import Dataset

In [26]:
class TitanicDataset(Dataset):
    def __init__(self, data_root, training=False, transform=None):
        self.data_root = data_root
        self.training = training
        self.transform = transform
        
        if training:
            file_name = 'train.csv'
        else:
            file_name = 'test.csv'
            
        self.dataset = pd.read_csv(os.path.join(data_root, file_name))
        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):  
        pclass = np.array(self.dataset['Pclass'][idx], dtype=np.float64)
        name = self.dataset['Name'][idx] # not used
        sex = np.array(pd.get_dummies(self.dataset['Sex']).iloc[idx], dtype=np.float64)
        age = np.array(self.dataset['Age'][idx] if not np.isnan(self.dataset['Age'][idx]) else -1, dtype=np.float64)
        sibsp = np.array(self.dataset['SibSp'][idx], dtype=np.float64)
        parch = np.array(self.dataset['Parch'][idx], dtype=np.float64)
        ticket = self.dataset['Ticket'][idx] # not used
        fare = np.array(self.dataset['Fare'][idx], dtype=np.float64)
        cabin = self.dataset['Cabin'][idx] # not used
        embarked = np.array(pd.get_dummies(self.dataset['Embarked']).iloc[idx], dtype=np.float64)
        
        if self.training:
            label = np.array(self.dataset['Survived'][idx], dtype=np.float64)
        else:
            label = None
        
        # feature: [paclass, female, male, age, sibsp, parch, fare, embarked(C), embarked(Q), embarked(S)]
        feature = np.concatenate((pclass, sex, age, sibsp, parch, fare, embarked), axis=None)
        sample = {'feature': feature, 'label': label}
        if self.transform:
            return self.transform(sample)
        else:
            return sample

In [29]:
dataset = TitanicDataset(data_root=config['data_root'], training=False)
train_set, valid_set = train_valid_split(dataset, 0.1)
len(train_set), len(valid_set)

(377, 41)

# Data Loader

In [1]:
from torch.utils.data import DataLoader

# Network

# Loss

# Metrics

# Trainer

# Train